In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import re 
import tensorflow as tf 

In [ ]:
# xây dựng trình tối ưu hóa cho mô hình Bert 
def create_optimizer(loss , init_lr, num_train_steps , num_warmup_steps, use_tpu):
    """Creates an optimizer training op."""
    # Khởi tạon một biến toàn bộ đại diện cho toàn bộ bước thời gian 
    global_step = tf.train.get_or_create_global_step()

    # khởi tạo một giá trị là lr là một giá trị float value = lr 
    learning_rate = tf.constant(value=init_lr , shape=[] , dtype=tf.float32)

    # Thực hiện các tuyến tính phân dã cho tỷ lệ học tập 
    learning_rate = tf.train.polynomial_decay(
        learning_rate , global_step , num_train_steps , end_learning_rate =0.0,
        power=1.0, cycle = False 
    )

    # làm nóng tuyến tính .. nếu như toàn bộ các bước thời gian < số lượn bước khởi động 
    # thì tỷ lêh học tập sẽ là 'global/num_warup_steps , * init_lt ' . 
    # đầu tiên ta kiểm tra xem num_warup_steps có tồn tại hay không 
    if num_warmup_steps: 
        # gán toàn bộ các bước thời gia là một giá trị int được biểu thị bởi Global_step 
        global_steps_int = tf.cast(global_step, dtype=tf.int32 )
        # tương tự lấy ra kết quả là số lượng bước khởi động sử dụng tf.constant để giá trị không 
        # bị thay đổi 
        warmup_steps_int = tf.constant(num_warmup_steps , dtype = tf.int32)

        # từ 2 giá trị là global_steps_intvà warmup_steps_int ta chuyển 2 giá trị thành float 
        global_steps_float = tf.cast(global_steps_int, tf.float32)
        warmup_steps_float = tf.cast(warmup_steps_int, tf.float32)

        # sau đó tính tỷ lệ hoàn thành của quá trình warmup và tỷ lệ học tập trong quá trình warmup 
        warmup_percent_done = global_steps_float / warmup_steps_float
        warmup_learning_rate = init_lr * warmup_percent_done

        # Cuối cùng nó kiểm tra xem global_steps có nhỏ hơn warmup_steps_int hay không
        # lưu kết qủa vào biến is_warmup có giá trị 1 nếu trong quá trình warup và o nếu 0 
        is_warmup = tf.cast(global_steps_int < warmup_steps_int , tf.float32)

        # và nó cũng cập nhật biến laearning_rate
        learning_rate = (
            (1.0 - is_warmup) * learning_rate + is_warmup * warmup_learning_rate)

        # Thiết lập trình tối ưu hóa 
        # sử dụng AdamWeightDecayOptimizer 
        optimizer = AdamWeightDecayOptimizer(
                learning_rate=learning_rate,
                weight_decay_rate=0.01,
                beta_1=0.9,
                beta_2=0.999,
                epsilon=1e-6,
                exclude_from_weight_decay=["LayerNorm", "layer_norm", "bias"]
                )

        # kiểm tra xem mô hình vó đang sử dụng Tpu không 
        # Nếu có thì ta sử dụng tpu để tối ưu hóa mô hình 
        if use_tpu : 
            optimizer = tf.contrib.tpu.CrossShardOptimizer(optimizer)
        

        # Lấy ra danh sách các biến giá trị có thể được huấn luyên từ mô hình và gá cho 
        # Tvar
        tvars = tf.trainable_variable()
        # và lấy ra kết quả của gradients 
        grads = tf.gradients(loss , tvars)

        # đầu tiên sử dụng hàm tf.clip_by_global_norm để cắt bớt các gradient của các biến 
        # có thể huấn luyện theo tỷ lệ tổng cac chuẩn của chúng . Mục đích là để tránh tạo các đối tượng 
        # gradient bùng nổ 
        (grads, _) = tf.clip_by_global_norm(grads, clip_norm=1.0)


        # sử dụng optimizer.apply_gradient để áp dụng các gradient đã cắt ớt cho các biến có thể 
        # được huấn luyện và cập nhật các giá trị global tại thời điểm đào tạo 
        train_op = optimizer.apply_gradients(
            zip(grads, tvars), global_step=global_step)
        
        # nó tạo một biến mới new_global_step bằng cách cộng global_step với 1,
        # và gán lại giá trị của global_step bằng new_global_step. Điều này là cần thiết vì lớp AdamWeightDecayOptimizer
        # không tự động cập nhật global_step khi áp dụng các gradient
        new_global_step = global_step + 1
        train_op = tf.group(train_op, [global_step.assign(new_global_step)])
        return train_op
    

class AdamWeightDecayOptimizer(tf.train.Optimizer):
    """A basic Adam optimizer that includes "correct" L2 weight decay."""

    def __init__(self,
                learning_rate,
                weight_decay_rate=0.0,
                beta_1=0.9,
                beta_2=0.999,
                epsilon=1e-6,
                exclude_from_weight_decay=None,
                name="AdamWeightDecayOptimizer"):
         
        """Constructs a AdamWeightDecayOptimizer."""
        super(AdamWeightDecayOptimizer, self).__init__(False, name)
        self.learning_rate = learning_rate
        self.weight_decay_rate = weight_decay_rate
        self.beta_1 = beta_1
        self.beta_2 = beta_2
        self.epsilon = epsilon
        self.exclude_from_weight_decay = exclude_from_weight_decay

    # Xây dựng phương thức áp dụng gradient cho các biến có thể được huấn luyên 
    def apply_gradients(self, grads_and_vars , global_step =None , name=None):
        """See base class"""
        # tạo một danh sách assigments để lưu trữ các kết quả của phép gán cập nhật tham số 
        assignments = []
        # duyêth qua danh sách gradient của tham số và kết quả tham số 
        for (grad , param) in grads_and_vars: 
            # nếu 1 trong 2 giá trị = None thì bỏ qua cặp này 
            if grad is None or param is None: 
                continue 

            # sau đó lấy ra tên của tham số và gán cho param name 
            param_name = self._get_variable_name(param.name)

            # khởi tạo 2 biến m và vcho mỗi tham số param , có cùng kích thước và kiểu dữ liệu với 
            # params  , m và v là các ước lượng trug bình động của gradinet (moment) và bình phương gradient 
            # của param 

            m = tf.get_variable(
                name=param_name + "/adam_m",
                shape=param.shape.as_list(),
                dtype=tf.float32,
                trainable=False,
                initializer=tf.zeros_initializer())
            v = tf.get_variable(
                name=param_name + "/adam_v",
                shape=param.shape.as_list(),
                dtype=tf.float32,
                trainable=False,
                initializer=tf.zeros_initializer())

            # cập nhật giá trị m và v theo công thức chuẩn Adam 
            # next_m = beta_1 * m + (1 - beta_1) * grad
            # next_v = beta_2 * v + (1 - beta_2) * grad^2
            next_m = (
                tf.multiply(self.beta_1, m) + tf.multiply(1.0 - self.beta_1, grad))
            next_v = (
                tf.multiply(self.beta_2, v) + tf.multiply(1.0 - self.beta_2,
                                                    tf.square(grad)))
            
            # tính toán giá trị cập nhật cho tham số params theo công thức chuẩn của Adam 
            # update  = next_m / sqrt (next_v) + epsilon 

            update = next_m / (tf.sqrt(next_v) + self.epsilon)

            # Kiểm tra xem tham số param có cần áp dụng suy giảm trọng số (weight decay) hay không bằng
            #  cách gọi hàm _do_use_weight_decay. Nếu có, thì cộng thêm một số lượng tỷ lệ với trọng số
            #  param vào giá trị cập nhật. Đây là một kỹ thuật
            #  để giảm thiểu độ phức tạp của mô hình và tránh hiện tượng quá khớp (overfitting).
            if self._do_use_weight_decay(param_name):
                update += self.weight_decay_rate * param
            
            # Nhân giá trị cập nhật với tốc độ học (learning rate), 
            # một tham số điều khiển bước nhảy của thuật toán tối ưu hóa.
            update_with_lr = self.learning_rate * update
            # Trừ giá trị cập nhật với tốc độ học ra khỏi tham số param để được tham số mới next_param.
            next_param = param - update_with_lr

            # Thêm các phép gán next_param, next_m và next_v vào danh sách assignments
            assignments.extend(
                [param.assign(next_param),
                m.assign(next_m),
                v.assign(next_v)])

        # Trả về kết quả của hàm group trong TensorFlow, để nhóm các phép gán lại thành
        # một đồ thị tính toán duy nhất. Tham số name là tên của đồ thị, nếu có.   
        return tf.group(*assignments, name=name)


    def _do_use_weight_decay(self, param_name):
        """Whether to use L2 weight decay for `param_name`."""
        if not self.weight_decay_rate:
            return False
        if self.exclude_from_weight_decay:
            for r in self.exclude_from_weight_decay:
                if re.search(r, param_name) is not None:
                    return False
        return True

    def _get_variable_name(self, param_name):
        """Get the variable name from the tensor name."""
        m = re.match("^(.*):\\d+$", param_name)
        if m is not None:
            param_name = m.group(1)
        return param_name